[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gdslab/d2spy/blob/main/docs/guides/notebooks/02_accessing_your_workspace.ipynb)
[![Jupyter Notebook](https://img.shields.io/badge/Open%20in%20JuypterHub%20-%20%233776AB?logo=jupyter&logoColor=%23F37626&labelColor=%23F5F5F5)](https://lab.d2s.org/hub/user-redirect/lab/tree/tutorials/02_accessing_your_workspace.ipynb)

# Accessing your workspace projects, flights, and data products
*This guide will walk you through the steps for requesting data from a Data to Science (D2S) instance.*

To get started, you will need to import the `Workspace` module.

In [ ]:
# Uncomment and run the following line if working out of Google Colab
# !pip install d2spy

In [ ]:
from datetime import date

from d2spy.workspace import Workspace

All of your D2S data can be accessed through a D2S "workspace." The [`Workspace`](https://py.d2s.org/workspace/) module's [`connect`](https://py.d2s.org/workspace/#d2spy.workspace.Workspace.connect) method can be used to login to a D2S instance and connect to your workspace in one go. Behind the scenes, the [`Auth`](https://py.d2s.org/auth/) module will be used to handle authenticating with D2S and requesting an authorization token. You will need to provide `connect` with the URL to your D2S instance and enter your password when prompted.

In [ ]:
# Example of connecting to a workspace for a local D2S instance
workspace = Workspace.connect("http://localhost:8000", "yourD2Semail@example.com")

The Workspace [`get_projects`](https://py.d2s.org/workspace/#d2spy.workspace.Workspace.get_projects) method will retrieve a collection of the projects your account can currently access on the D2S instance.

In [ ]:
# Get list of all your projects
projects = workspace.get_projects()
# Print title of first project (if one exists)
if len(projects) > 0:
    print(projects[0])
else:
    print("Please create a project before proceeding with this guide.")

The `projects` variable is a [`ProjectCollection`](https://py.d2s.org/project_collection/). The collection can be filtered by either the project descriptions or titles using the methods [`filter_by_title`](https://py.d2s.org/project_collection/#d2spy.models.project_collection.ProjectCollection.filter_by_description) or [`filter_by_name`](https://py.d2s.org/project_collection/#d2spy.models.project_collection.ProjectCollection.filter_by_title).

In [ ]:
# Example of creating new collection of only projects with the keyword "Test" in the title
filtered_projects = projects.filter_by_title("Test")
print(filtered_projects)

The Project [`get_project_boundary`](https://py.d2s.org/project/#d2spy.models.project.Project.get_project_boundary) method will retrieve a GeoJSON object of the project boundary.

In [ ]:
# Get project boundary as Python dictionary in GeoJSON structure
project_boundary = projects[0].get_project_boundary()
print(project_boundary)

The Project [`get_flights`](https://py.d2s.org/project/#d2spy.models.project.Project.get_flights) method will retrieve a list of the flights associated with a project.

In [ ]:
# Get list of all flights for a project
flights = projects[0].get_flights()
# Print first flight object (if one exists)
if len(flights) > 0:
    print(flights[0])

The `flights` variable is a [`FlightCollection`](https://py.d2s.org/flight_collection/). The collection can be filtered by the acquisition date using the method [`filter_by_date`](https://py.d2s.org/flight_collection/#d2spy.models.flight_collection.FlightCollection.filter_by_date). This method will return all flights with an acquisition date bewteen the provided start and end dates.

In [ ]:
# Example of creating new collection of only flights from May 2024 - June 2024
filtered_flights = flights.filter_by_date(start_date=date(2024, 5, 1), end_date=date(2024, 6, 30))
print(filtered_flights)

A [`FlightCollection`](https://py.d2s.org/flight_collection/) can also be filtered by sensor using the method [`filter_by_sensor`](https://py.d2s.org/flight_collection/#d2spy.models.flight_collection.FlightCollection.filter_by_sensor). This method will return all flights with a sensor that matches the provided search term.

In [ ]:
filtered_flights = filtered_flights.filter_by_sensor("rgb")
print(filtered_flights)

The Flight [`get_data_products`](https://py.d2s.org/flight/#d2spy.models.flight.Flight.get_data_products) method will retrieve a list of the data products associated with a flight.

In [ ]:
# Get list of data products from a flight
data_products = flights[0].get_data_products()
# Print the url for the first data product (if one exists)
print(data_products[0].url)

The `data_products` variable is a [`DataProductCollection`](https://py.d2s.org/data_product_collection/). The collection can be filtered by the data type using the method [`filter_by_data_type`](https://py.d2s.org/data_product_collection/#d2spy.models.data_product_collection.DataProductCollection.filter_by_data_type). This method will return all data products that match the requested data type.

In [ ]:
# Example of creating new collection of data products with the "ortho" data type
filtered_data_products = data_products.filter_by_data_type("ortho")
print(filtered_data_products)

We can access the band info for a [`DataProduct`](https://py.d2s.org/data_product/) using the [`get_band_info`](https://py.d2s.org/data_product/#d2spy.models.data_product.DataProduct.get_band_info) method. The [`update_band_info`](https://py.d2s.org/data_product/#d2spy.models.data_product.DataProduct.update_band_info) method can be used to assign a new `description` to one or more bands. Only the band `description` values can be changed. The band `name` values must remain the same.

Example band info returned from `get_band_info` for a 4-band orthomosaic:
```python
[
    {'name': 'b1', 'description': 'Red'},
    {'name': 'b2', 'description': 'Green'},
    {'name': 'b3', 'description': 'Blue'},
    {'name': 'b4', 'description': 'Alpha'}
]
```

In [ ]:
# Print band names and descriptions for a data product
band_info = filtered_data_products[0].get_band_info()
print(band_info)

# Update "b1" and "b3" band descriptions
band_info[0]["description"] = "Blue"
band_info[2]["description"] = "Red"
filtered_data_products[0].update_band_info(band_info)

These are all the methods you need to know for viewing what data your account can access on a D2S instance. Once finished exploring your data, you can revoke your authorization session by logging out.

In [ ]:
# Removes access token from future requests
workspace.logout()